## Model Training

In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/waste-detection-using-yoloV5'

In [4]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    yolo_model_gitgub_url: str
    num_classes: int
    pretrained_model_name: str
    image_size: int
    batch_size: int
    epochs: int
    required_files: list
    dataset_dir: Path

In [5]:
from wasteDetection.constants import *
from wasteDetection.utils import create_directories, read_yaml, save_yaml

In [6]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        create_directories([self.config.model_training.root_dir])
        data_file_path = Path(self.config.data_ingestion.dataset_dir, 'data.yaml')
        data_config = read_yaml(data_file_path)

        model_training_config = ModelTrainingConfig(
            root_dir = Path(self.config.model_training.root_dir),
            yolo_model_gitgub_url = self.config.model_training.yolo_model_gitgub_url,
            num_classes =  int(data_config.nc),
            pretrained_model_name = self.params.PRETRAINED_MODEL_NAME,
            image_size = self.params.IMAGE_SIZE,
            batch_size = self.params.BATCH_SIZE,
            epochs = self.params.EPOCHS,
            required_files = self.params.REQUIRED_FILES,
            dataset_dir = self.config.data_ingestion.dataset_dir
        )

        return  model_training_config


In [7]:
import subprocess
import shutil
import yaml

In [8]:
# Component
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def _get_yolo_model_from_gitgub(self):
        self.repo_name = self.config.yolo_model_gitgub_url.split('/')[-1].split('.')[0]
        if (not os.path.exists(self.repo_name)):
            subprocess.call(["git", "clone", self.config.yolo_model_gitgub_url])
            shutil.rmtree(os.path.join(self.repo_name,'.git'))

    def _make_custom_model_config(self):
        pretrained_model_path = Path(f'yolov5/models/{self.config.pretrained_model_name}.yaml')
        
        with open(pretrained_model_path) as f:
            pretrained_model_config = yaml.safe_load(f)
        
        pretrained_model_config['nc'] = self.config.num_classes
        custom_model_config_path = Path(f'yolov5/models/custom_{self.config.pretrained_model_name}.yaml')
        
        with open(custom_model_config_path, 'w') as f:
                yaml.dump(pretrained_model_config, f)

    def _train_model(self):
        for require_file in self.config.required_files:
            require_file_path = os.path.join(self.config.dataset_dir, require_file)
            destination_file_path = os.path.join(self.repo_name, require_file)
            if not os.path.exists(destination_file_path):
                if os.path.isdir(require_file_path):
                    shutil.copytree(require_file_path, destination_file_path)
                else:
                    shutil.copy(require_file_path, destination_file_path)
                
        os.system(f"cd {self.repo_name}/ && python train.py --img {self.config.image_size} --batch {self.config.batch_size} --epochs {self.config.epochs} --data data.yaml --cfg models/custom_{self.config.pretrained_model_name}.yaml --weights '{self.config.pretrained_model_name}.pt' --name {self.config.pretrained_model_name}_results --cache")
        
    def _model_training_post_processing(self):
        os.system(f"cp {self.repo_name}/runs/train/{self.config.pretrained_model_name}_results/weights/best.pt {self.config.root_dir}/")
        required_files = self.config.required_files
        required_files.append('runs')

        for require_file in required_files:
            require_file_path = os.path.join(self.repo_name,require_file)
            if os.path.exists(require_file_path):
                if os.path.isdir(require_file_path):
                    shutil.rmtree(require_file_path)
                else:
                    os.remove(require_file_path)
        

    def initiate_model_training(self):
        try:
            self._get_yolo_model_from_gitgub()
            self._make_custom_model_config()
            self._train_model()
            self._model_training_post_processing()
        except Exception as e:
            raise e


In [9]:
# Pipeline
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config)
    model_training.initiate_model_training()

except Exception as e:
    raise(e)

2023-11-23 00:46:12,835 : wasteDetection.logger - INFO - YAML file: config/config.yaml loaded successfully
2023-11-23 00:46:12,839 : wasteDetection.logger - INFO - YAML file: params.yaml loaded successfully
2023-11-23 00:46:12,840 : wasteDetection.logger - INFO - created directory at: artifacts
2023-11-23 00:46:12,841 : wasteDetection.logger - INFO - created directory at: artifacts/model_training
2023-11-23 00:46:12,843 : wasteDetection.logger - INFO - YAML file: artifacts/data_ingestion/dataset/data.yaml loaded successfully
Cloning into 'yolov5'...
train: weights=yolov5s.pt, cfg=models/custom_yolov5s.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


100%|██████████| 14.1M/14.1M [00:06<00:00, 2.16MB/s]


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3,